In [15]:
import os
import pandas as pd
import numpy as np
from IPython.display import display
from typing import TypedDict, List, Literal, Annotated, Dict, Union
from pydantic import BaseModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.types import Command
import json

from prompt.prompts import *
from utils.vector_handler import load_vectorstore

from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-4o", openai_api_key=openai_api_key, temperature=0)


In [2]:
# 환경 설정
from dotenv import load_dotenv
import os
import pandas as pd
from ai_agent_v2 import DataAnayticsAssistant

# OpenAI API 키 로드
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

PROCESSED_DATA_PATH = "../output/stage1/processed_data_info.xlsx"
mart_name = "cust_intg"
def load_processed_data_info():
    """사전에 분석된 데이터 정보 로드"""
    if not os.path.exists(PROCESSED_DATA_PATH):
        return None
    else:
        # 모든 시트 로드
        return pd.read_excel(PROCESSED_DATA_PATH, sheet_name=mart_name)

# ✅ Streamlit 실행 시 데이터 로드
mart_info = load_processed_data_info()

# 어시스턴트 초기화
assistant = DataAnayticsAssistant(openai_api_key, mart_info)

df = pd.read_pickle(f'../data/{mart_name}.pkl')

# 데이터프레임 설정
assistant.set_active_mart(df, mart_name)
llm = ChatOpenAI(model="gpt-4o", openai_api_key=openai_api_key, temperature=0)


✅ 그래프 컴파일 완료
✅ 1개의 데이터프레임이 성공적으로 설정되었습니다.
📊 데이터마트 이름: cust_intg
🔹 데이터 크기: 120000행 x 123열


In [3]:
class Planner:
    def __init__(self, llm):
        self.llm = llm

    def parse_question(self, user_request: str):
        """사용자의 질문을 절차 리스트로 변환"""
        prompt = ChatPromptTemplate.from_messages([
            ("system", """
    다음 자연어 질문을 분석하여 실행 가능한 분석 태스크 리스트로 변환해 주세요. 

    질문:
    "주어진 데이터셋의 컬럼별 데이터 타입, 결측치 개수, 기초 통계량(평균, 중앙값, 표준편차 등)을 분석 하고, 
    결측치 처리를 하고, CMIP컬럼과 가장 상관계수가 높은 변수를 하나 선택하고, 
    선택된 변수와 CMIP컬럼의 상관관계를 분석하고, 
    선택된 변수를 명시하며 CMIP컬럼의 정규성 검정을 하며, 
    정규성을 만족하든 만족하지 않든 그것에 맞는 회귀분석을 해주세요. 
    단, 데이터는 1000건만 이용하세요."

    출력 형식 (JSON):
    {{
        "tasks": [
            {{"step": 1, "task": "데이터 샘플링", "description": "데이터를 1000건만 사용하도록 샘플링"}},
            {{"step": 2, "task": "기초 분석", "description": "컬럼별 데이터 타입, 결측치 개수, 기초 통계량(평균, 중앙값, 표준편차 등) 확인"}},
            {{"step": 3, "task": "결측치 처리", "description": "결측치 0 또는 기타 값으로 대체"}},
            {{"step": 4, "task": "상관분석", "description": "CMIP 컬럼과 가장 상관계수가 높은 변수 선택"}},
            {{"step": 5, "task": "상관계수 분석", "description": "선택된 변수와 CMIP 컬럼 간의 상관계수 계산"}},
            {{"step": 6, "task": "정규성 검정", "description": "CMIP 컬럼의 정규성 검정 수행"}},
            {{"step": 7, "task": "회귀분석", "description": "정규성을 만족하면 선형 회귀, 만족하지 않으면 강건 회귀 적용"}}
        ]
    }}
    """),
            ("user", "질문: {user_request}")
        ])
        chain = prompt | self.llm
        response = chain.invoke({"user_request": user_request})
        return response

    def execute(self, user_request: str):
        """Planner 실행"""
        analysis_steps = self.parse_question(user_request)
        return analysis_steps


In [6]:

openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4o", openai_api_key=openai_api_key, temperature=0)
planner = Planner(llm)
result = planner.execute("""
주어진 데이터셋의 컬럼별 데이터 타입, 결측치 개수, 기초 통계량(평균, 중앙값, 표준편차 등)을 분석한 뒤에, 결측치 처리를 하고,
CMIP컬럼과 가장 상관계수가 높은 변수를 하나 선택하고, 선택된 변수와 CMIP컬럼의 상관관계를 분석하고, 
선택된 변수를 명시하며 CMIP컬럼의 정규성 검정을 하며, 정규성을 만족하든 만족하지 않든 그것에 맞는 회귀분석을 해주세요. 
단, 데이터는 1000건만 이용하세요.
""")
input_text =  result.content.split('```json')[1].split('```')[0].replace('\n', '')
parsed_list = json.loads(input_text)
parsed_list



In [ ]:
result = planner.execute("""각 변수의 데이터 타입을 검사하고, 잘못된 데이터 타입이 있는 경우 적절한 변환 방법을 제안해 주세요.""")
input_text =  result.content.split('```json')[1].split('```')[0].replace('\n', '')
parsed_list = json.loads(input_text)
parsed_list

In [ ]:
result = planner.execute("""
데이터의 카디널리티(Cardinality)가 높은 범주형 변수를 탐색하고, 이를 차원 축소해주세요.""")
input_text =  result.content.split('```json')[1].split('```')[0].replace('\n', '')
parsed_list = json.loads(input_text)
parsed_list

In [ ]:
result = planner.execute("""성별로 데이터를 그룹화하고, 금융 거래 금액과 관련된 컬럼들의 평균을 계산해 주세요.""")
input_text =  result.content.split('```json')[1].split('```')[0].replace('\n', '')
parsed_list = json.loads(input_text)
parsed_list

In [25]:
# ✅ 경로 결정용 라우터
class Router(BaseModel):
    next: Literal["Analytics", "General", "Knowledge", "Generate_Code", "Execute_Sample", "Regenerate_Code", "Execute_Full", 
                  "Save_Data", "Retrieve_Data", "Insight_Builder", "Chart_Builder", "Report_Builder", "__end__"]

def supervisor(step) -> Command:
    """Planner에서 전달받은 분석 리스트를 처리하여 Supervisor가 분류"""
    print("="*100)
    print(f"👨‍💼 Supervisor 단계: {step}")

    # Planner가 분석 절차 리스트를 반환했다고 가정

    # 각 분석 절차를 Analytics로 분류
    prompt = ChatPromptTemplate.from_messages([
            ("system", PROMPT_SUPERVISOR),
            ("user", " user_request:\n{user_request}\n\n")
    ])
    chain = prompt | llm.with_structured_output(Router)
    response = chain.invoke({"user_request": step})
    print(f"🏃🏿‍➡️ 다음 단계: {response.next}")

    
for i, x in enumerate(parsed_list):
    supervisor(x)


In [24]:
df.head()

,고객ID,나이,성별,수익자여부,CB신용평점,CB신용등급,두낫콜여부,운전코드명,성별코드,피보험자여부,...,변액보유여부,변액최대납입회차,변액유지계약수,변액기납입보험료,변액종신CMIP,변액종신보유여부,변액종신최대납입회차,변액종신유지계약수,변액종신기납입보험료,기준년월
0,25226,60대,여성,1,NaN,NaN,1.0,승용차(자가용),02,1,...,0,11.09,0.0,6424.05,69074.36,0,3.15,0.0,539090.38,202405
1,95256,40대,남성,1,NaN,6.0,0.0,화물차(자가용),02,1,...,0,3.98,0.0,9592.97,2023.21,0,34.68,0.0,3338509.65,202405
2,14751,40대,여성,1,NaN,NaN,1.0,NaN,02,1,...,0,4.06,0.0,22093.94,11006.27,0,9.01,0.0,1356509.39,202405
3,4478,50대,여성,1,NaN,6.0,0.0,운전안함,01,1,...,0,8.71,0.0,6071.17,6473.26,0,1.02,0.0,16478801.52,202405
4,100000,50대,여성,0,245.26,NaN,NaN,운전안함,02,1,...,0,1.94,0.0,16569.67,2852.47,0,17.67,0.0,373537.44,202405


In [27]:
import json
example_json = {
    "분석_계획": [
        {
            "주제": "기초 데이터 분석",
            "설명": "데이터의 기본 통계량 확인 및 결측치 처리",
            "value": df.describe().round(2)
        },
        {
            "주제": "고객 특성별 거래 패턴", 
            "설명": "성별/연령대에 따른 금융 거래 행태 분석",
            "value": ["성별", "연령대", "거래금액", "거래건수"]
        },
        {
            "주제": "세그먼트 분석",
            "설명": "고객 세그먼트별 거래 지표 분석", 
            "value": {
                "segments": ["VIP", "GOLD", "SILVER"],
                "metrics": ["avg_amount", "transaction_count"]
            }
        },
        {
            "주제": "이상치 분석",
            "설명": "데이터 내 이상치 탐지 및 처리",
            "value": {
                "outlier_columns": ["거래금액", "거래건수"],
                "method": "IQR"
            }
        },
    ]
}

if example_json:
    for key, value in example_json.items():
        print(f"**{key}**:")
        print(value)
else:
    print("🔍 분석 결과 데이터가 없습니다.")

**분석_계획**:
[{'주제': '기초 데이터 분석', '설명': '데이터의 기본 통계량 확인 및 결측치 처리', 'value':             고객ID      수익자여부    CB신용평점    CB신용등급     두낫콜여부     피보험자여부  \
count  120000.00  120000.00  14118.00  18594.00  98544.00  120000.00   
mean    43126.17       0.89    316.21      4.84      0.36       0.85   
std     33176.95       0.31    280.35     13.31      0.48       0.36   
min         1.00       0.00      0.00      0.00      0.00       0.00   
25%     14297.25       1.00     95.16      1.00      0.00       1.00   
50%     34112.50       1.00    233.25      2.00      0.00       1.00   
75%     69490.25       1.00    457.66      5.00      1.00       1.00   
max    100000.00       1.00   1049.57     99.00      1.00       1.00   

            보험연령  직업위험등급코드      시도코드  방카채널Affluent고객여부  ...     변액보유여부  \
count  120000.00  34692.00  95652.00         120000.00  ...  120000.00   
mean       45.36      4.72      6.73              0.00  ...       0.02   
std        12.93      7.85      5.04              0.03 

In [31]:
analysis_results 

{'분석주제': '컬럼별 고유값 개수 분석 및 고유값이 많은 컬럼 처리 방안',
 '설명': '데이터프레임의 각 컬럼에 대해 고유값의 개수를 분석하고, 고유값이 50개를 초과하는 컬럼에 대해 처리 방안을 제안합니다.',
 '결과값': {'고유값 개수': {'대출가능금액합계': 119994,
   '연금기납입보험료': 119993,
   '변액종신기납입보험료': 119992,
   '기납입보험료합계': 119965,
   '저축가입납입보험료': 119964,
   'CMIP': 119962,
   '보험계약대출잔액합계': 119938,
   '변액CMIP': 119203,
   '미납보험료합계': 118886,
   '변액기납입보험료': 117933,
   '변액종신CMIP': 117901,
   '저축CMIP': 115456,
   '누적연금지급금액': 68313,
   '누적해약환급금지급금액': 67978,
   '누적중도보험금지급금액': 67562,
   'CM채널CMIP': 57843,
   '자사설계사채널CMIP': 39305,
   '전사최초계약경과월수': 21700,
   '전사최종계약경과월수': 20789,
   '일반종신최대납입회차': 17717,
   '고객ID': 15474,
   'GA채널CMIP': 14956,
   'CB신용평점': 12570,
   '당월보험료자동대출잔액': 10036,
   '변액종신최대납입회차': 6011,
   '누적콜센터상담건수': 5947,
   '연금최대납입회차': 4619,
   '기타채널CMIP': 3461,
   '아웃바운드채널CMIP': 3218,
   '누적부정반응건수': 2170,
   '변액최대납입회차': 1900,
   '누적보험금지급건수': 1234,
   '저축최대납입회차': 1229,
   '가입특약수합계': 1015,
   '모집설계사수': 429,
   '수금설계사수': 395,
   '일반종신누적가입건수': 343,
   '당월자사설계사채널계약이탈건수': 338,
   '교차채널CMI

In [30]:
import pandas as pd
import numpy as np
import sys, os
import io
from IPython.display import display
from typing import TypedDict, List, Literal, Annotated, Dict, Union
from pydantic import BaseModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.types import Command
import json

from prompt.prompts import *
from utils.vector_handler import load_vectorstore
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-4o", openai_api_key=openai_api_key, temperature=0)

mart_name = "cust_intg"
df = pd.read_pickle(f'../data/{mart_name}.pkl')

# 생성형 AI가 생성한 코드를 실행하고 출력을 저장하는 함수
def execute_code_with_capture(code):
    # 표준 출력을 가로채기 위해 StringIO 사용
    captured_output = io.StringIO()
    original_stdout = sys.stdout  # 원래 표준 출력 저장
    sys.stdout = captured_output  # 표준 출력 변경

    # 안전한 실행 환경을 위한 locals() 네임스페이스 사용
    safe_locals = { }

    try:
        exec(code, globals(), safe_locals)  # **제한된 네임스페이스에서 실행**
    except Exception as e:
        captured_output.write(f"Error: {str(e)}\n")  # 에러 메시지 출력

    # 표준 출력을 원래대로 복원
    sys.stdout = original_stdout

    analysis_results = safe_locals.get("analysis_results", {})

    print(f'analysis_results:\n {analysis_results}')
    return captured_output.getvalue(), analysis_results

PROMPT_GENERATE_CODE = """
사용자 요청에 대한 Python 코드를 생성해주세요.
사용할 데이터프레임은 반드시 'df' 변수로 제공되며, 새롭게 예제 데이터를 생성하지 마세요.

다음 규칙을 반드시 따라주세요:
1. **제공된 데이터프레임에 대한 처리를 해주는 코드를 생성해주세요.**
2. **결과 저장 형식 (`analysis_results`)**
   - 분석 결과를 json 형태의 'analysis_results' 변수에 저장해주세요.
   - 'analysis_results'(json)은 '분석주제', '설명', '결과값'라는 각각의 key와 해당하는 value들을 채워주세요.
   - **집계성 데이터(aggregated data)**는 전체 데이터를 저장하고, 반드시 `print()`로 출력하세요.
   - **비집계성 데이터(non-aggregated data)**는 `head()` 적용 후 `round(2)` 처리한 데이터를 저장하세요.
   - **모든 수치형 데이터는 `round(2)`를 적용**한 후 저장하세요.
3. **코드만 제공하고, 추가 설명이나 주석을 포함하지 마세요.**
"""


prompt = ChatPromptTemplate.from_messages([
            ("system", PROMPT_GENERATE_CODE),
            ("user", " user_request:\n{user_request}\n\n"),
    ])
chain = prompt | llm
response = chain.invoke({
    "user_request": f"""컬럼별 고유값 개수를 분석하고, 지나치게 많은 고유값을 가진 컬럼이 있는 경우 이를 다루는 최적의 방법을 제안해 주세요.""",
    }
)
print(response.content + "")
ai_generated_code = response.content.split('```python')[1].split('```')[0]
output, analysis_results = execute_code_with_capture(ai_generated_code)
exec(response.content.split('```python')[1].split('```')[0])
if analysis_results:
    for key, value in analysis_results.items():
        print(f"**{key}**:")
        print(value)
else:
    print("🔍 분석 결과 데이터가 없습니다.")

```python
unique_counts = df.nunique().sort_values(ascending=False)
high_cardinality_columns = unique_counts[unique_counts > 50]

analysis_results = {
    "분석주제": "컬럼별 고유값 개수 분석 및 고유값이 많은 컬럼 처리 방안",
    "설명": "데이터프레임의 각 컬럼에 대해 고유값의 개수를 분석하고, 고유값이 50개를 초과하는 컬럼에 대해 처리 방안을 제안합니다.",
    "결과값": {
        "고유값 개수": unique_counts.to_dict(),
        "고유값이 많은 컬럼": high_cardinality_columns.to_dict(),
        "처리 방안": "고유값이 많은 컬럼은 범주형 데이터로 변환하거나, 필요에 따라 차원 축소 기법을 사용할 수 있습니다."
    }
}

print(unique_counts)
```
analysis_results:
 {'분석주제': '컬럼별 고유값 개수 분석 및 고유값이 많은 컬럼 처리 방안', '설명': '데이터프레임의 각 컬럼에 대해 고유값의 개수를 분석하고, 고유값이 50개를 초과하는 컬럼에 대해 처리 방안을 제안합니다.', '결과값': {'고유값 개수': {'대출가능금액합계': 119994, '연금기납입보험료': 119993, '변액종신기납입보험료': 119992, '기납입보험료합계': 119965, '저축가입납입보험료': 119964, 'CMIP': 119962, '보험계약대출잔액합계': 119938, '변액CMIP': 119203, '미납보험료합계': 118886, '변액기납입보험료': 117933, '변액종신CMIP': 117901, '저축CMIP': 115456, '누적연금지급금액': 68313, '누적해약환급금지급금액': 67978, '누적중도보험금지급금액': 67562, 'CM채널CMIP': 57843, '자사설계사채널CMIP': 393

In [ ]:
import pandas as pd
import numpy as np
import sys, os
import io
from IPython.display import display
from typing import TypedDict, List, Literal, Annotated, Dict, Union
from pydantic import BaseModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.types import Command
import json

from prompt.prompts import *
from utils.vector_handler import load_vectorstore
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-4o", openai_api_key=openai_api_key, temperature=0)

mart_name = "cust_intg"
df = pd.read_pickle(f'../data/{mart_name}.pkl')

# 생성형 AI가 생성한 코드를 실행하고 출력을 저장하는 함수
def execute_code_with_capture(code):
    # 표준 출력을 가로채기 위해 StringIO 사용
    captured_output = io.StringIO()
    original_stdout = sys.stdout  # 원래 표준 출력 저장
    sys.stdout = captured_output  # 표준 출력 변경

    # 안전한 실행 환경을 위한 locals() 네임스페이스 사용
    safe_locals = { }

    try:
        exec(code, globals(), safe_locals)  # **제한된 네임스페이스에서 실행**
    except Exception as e:
        captured_output.write(f"Error: {str(e)}\n")  # 에러 메시지 출력

    # 표준 출력을 원래대로 복원
    sys.stdout = original_stdout

    analysis_results = safe_locals.get("analysis_results", {})

    print(f'analysis_results:\n {analysis_results}')
    return captured_output.getvalue(), analysis_results

PROMPT_GENERATE_CODE = """
사용자가 제공한 Python 코드에 대한 설명을 작성해 주세요.

설명은 아래 형식을 따라 주세요:
1. **코드 목적 (Objective)**
   - 코드가 수행하는 핵심 작업을 간략히 설명합니다. (1줄)

2. **코드 단계별 설명 (Step-by-step Explanation)**
   - 코드의 주요 단계별로 어떤 작업을 수행하는지 설명합니다. (1줄)

3. **출력 결과 설명 (Expected Output)**
   - 코드 실행 후 예상되는 출력 값을 출력 변수명과 함께 설명합니다.
"""

user_code = '''
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Assuming 'text_column' is the column with text data
text_column = '운전코드명'

# Initialize CountVectorizer with stop words
vectorizer = CountVectorizer(stop_words='english')

# Fit and transform the text data
text_data = df[text_column].fillna('')
X = vectorizer.fit_transform(text_data)

# Sum up the counts of each vocabulary word
word_counts = X.toarray().sum(axis=0)

# Create a DataFrame with words and their counts
word_freq_df = pd.DataFrame({
    'word': vectorizer.get_feature_names_out(),
    'count': word_counts
})

# Sort the DataFrame by count in descending order
word_freq_df = word_freq_df.sort_values(by='count', ascending=False)

# Store the results in the analysis_results dictionary
analysis_results = {
    'word_frequency': word_freq_df.round(2).head()
}

# Print the aggregated data
print(word_freq_df.round(2))
'''

prompt = ChatPromptTemplate.from_messages([
            ("system", PROMPT_GENERATE_CODE),
            ("user", " user_request:\n{user_request}\n\n"),
            ("user", " user_code:\n```python\n{user_code}\n\n"),
    ])
chain = prompt | llm
response = chain.invoke({
    "user_request": f"""텍스트 데이터가 포함된 컬럼에서 단어 빈도를 분석하고, 의미 없는 단어(Stopwords)를 제거하는 방법을 적용해 주세요.""",
    "user_code": user_code
    }
)
print(response.content + "")
# ai_generated_code = response.content.split('```python')[1].split('```')[0]
# output, analysis_results = execute_code_with_capture(ai_generated_code)
# exec(response.content.split('```python')[1].split('```')[0])
# if analysis_results:
#     for key, value in analysis_results.items():
#         print(f"**{key}**:")
#         print(value)
# else:
#     print("🔍 분석 결과 데이터가 없습니다.")

In [21]:
print("\n[분석 결과]")
for key, value in analysis_results.items():
    print(f"\n📊 {key}")
    print("-" * 50)
    print(value)


[분석 결과]

📊 data_types
--------------------------------------------------
고객ID            int64
나이             object
성별             object
수익자여부           int64
CB신용평점        float64
               ...   
변액종신보유여부        int64
변액종신최대납입회차    float64
변액종신유지계약수     float64
변액종신기납입보험료    float64
기준년월            int64
Length: 123, dtype: object

📊 missing_values
--------------------------------------------------
고객ID               0
나이                 0
성별                 0
수익자여부              0
CB신용평점        105882
               ...  
변액종신보유여부           0
변액종신최대납입회차         0
변액종신유지계약수          0
변액종신기납입보험료         0
기준년월               0
Length: 123, dtype: int64

📊 basic_stats
--------------------------------------------------
            고객ID      수익자여부    CB신용평점    CB신용등급     두낫콜여부     피보험자여부  \
count  120000.00  120000.00  14118.00  18594.00  98544.00  120000.00   
mean    43126.17       0.89    316.21      4.84      0.36       0.85   
std     33176.95       0.31    280.35     13.31  